In [1]:
%%capture
%pip install flask
%pip install pandas
%pip install gensim
%pip install nltk
%pip install torch
%pip install torchvision
%pip install tensorflow

In [2]:
import json
import csv
import random
import pandas as pd
import sklearn
import nltk
import torch
import torchvision
import tensorflow
import re
import string
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# DATA

In [3]:
file_path = 'Data/FEVER_Data/FEVER_train.jsonl'
FEVER_train = []
with open(file_path, 'r') as file:
    for line in file:
        data = json.loads(line)
        FEVER_train.append(data)

In [4]:
#liar plus data
liar_plus_train = pd.read_csv('Data/Liar_plus/train.tsv', delimiter='\t', header = None, quoting=csv.QUOTE_NONE)
liar_plus_train.columns = ["ID", "Json_File_ID", "Truth_Label", "Statement", "Subject", "Speaker", "Speakers_Job", "State", "Party",
                      "Barely_True_Counts", "False_Counts", "Half_True_Counts", "Mostly_True_Counts", "Pants_On_Fire_Counts", "Context_Venue_Location", "Justification"]
liar_plus_train = liar_plus_train.drop(columns='ID')
liar_plus_train

,Json_File_ID,Truth_Label,Statement,Subject,Speaker,Speakers_Job,State,Party,Barely_True_Counts,False_Counts,Half_True_Counts,Mostly_True_Counts,Pants_On_Fire_Counts,Context_Venue_Location,Justification
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"""Surovell said the decline of coal """"started w..."
2,324.json,mostly-true,"""Hillary Clinton agrees with John McCain """"by ...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,"""Obama said he would have voted against the am..."
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,"""The release may have a point that Mikulskis c..."
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"""Crist said that the economic """"turnaround sta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10295,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"""interview on """"The Colbert Report""""""",They compounded their error by combining full ...
10296,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview,"""Romney said that """"Obamacare means that for ..."
10297,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate,"""But that it leaves out important details and ..."
10298,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,"We checked the research and, quite frankly, fi..."


In [5]:
#clickbbait_data
clickbait = pd.read_csv("Data/Clickbait_Data/clickbait_data.csv")
clickbait

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1
...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",0
31996,"British Liberal Democrat Patsy Calton, 56, die...",0
31997,Drone smartphone app to help heart attack vict...,0
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",0


In [6]:
politifact_articles = pd.read_csv("Data/Politifact_Data/CSV/politifact_articles.csv")
politifact_articles = politifact_articles.drop(columns=['Unnamed: 0'])
politifact_articles

,Statement,Tldr_text_statements,Text
0,Did Mike Johnson work for a hate group? SPLC a...,[],"Since U.S. Rep. Mike Johnson, R-La., won the O..."
1,Ask PolitiFact: How many people on the terrori...,['The FBI’s terrorist watchlist is a federal d...,Some Republican lawmakers are flagging Hamas’ ...
2,Mike Johnson sought to overturn 2020 election....,"['In December 2020, U.S. Rep. Mike Johnson, R-...",After House Republicans unanimously voted for ...
3,Did Donald Trump make these 27 campaign promis...,[],A liberal group says former President Donald T...
4,Guerra de Israel-Hamas: Cómo se esparcieron re...,['Después de que el grupo militante palestino ...,Nota del Editor: Este artículo contiene refere...
...,...,...,...
2995,Congress blocked Obama's call for new gun laws...,[],It was an issue over which President Barack Ob...
2996,"Hate crimes, civil rights become law enforceme...",[],"President Barack Obama took office vowing to ""..."
2997,What would the impact be if the Affordable Car...,[],The Republican Congress and the incoming Trump...
2998,Barack Obama's top 25 campaign promises: How'd...,[],Editor’s note: PolitiFact will soon publish a ...


In [7]:
politifact_truth = pd.read_csv("Data/Politifact_Data/CSV/politifact_truthometer_df.csv")
politifact_truth = politifact_truth.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
politifact_truth

,Claimer,Statement,Truth_value,Tldr_text_statements,Text
0,Melissa Agard,"“Since 1981, the state Senate has only rejecte...",true,['The state Senate voted to fire eight of the ...,The Republican-controlled state Senate on Oct....
1,Instagram posts,“Lego donates model MRI kits to hospitals to h...,true,['A Lego employee and physicians at a Danish h...,"Undergoing a medical procedure, such as an MRI..."
2,Brian Krassenstein,It has been U.S. policy “for at least 79 years...,true,['Many airlines have canceled flights from Isr...,"Editor’s note, Oct. 18: After publication, we ..."
3,Mike Oliverio,"“A few years back,” Mitchell Stadium in Bluefi...",true,"['In a USA Today contest in 2019, Bluefield’s ...",As the 2023 high school football season got un...
4,Instagram posts,"""Beyonce paid $100K for Maryland Metro to run ...",true,"['Beyoncé’s Renaissance World Tour paid $100,0...","Washington, D.C.-area concertgoers flocking to..."
...,...,...,...,...,...
22060,Bloggers,"Obama says America is great, but let's ""try to...",pants-fire,[],"They have online names like fiberguy, dittohea..."
22061,Hillary Clinton,"Obama ""basically threatened to bomb Pakistan.""",pants-fire,[],"At the Democratic debate in Cleveland, Ohio, o..."
22062,Bloggers,"""Barack Obama loves Che Guevara.""",pants-fire,[],\nBarack Obama has been subjected to chain e-m...
22063,John McCain,"Obama ""suggested bombing Pakistan.""",pants-fire,[],"At a media availability in Columbus, Ohio, on ..."


In [8]:
#0 means fake, 1 means real
fakenews_data = pd.read_csv("Data/Kaggle_FakeTrue/Fake.csv")
realnews_data = pd.read_csv("Data/Kaggle_FakeTrue/True.csv")
fakenews_data["label"] = 0
realnews_data["label"] = 1

In [9]:
#0 means fake, 1 means real
WELfake_data = pd.read_csv("Data/Kaggle_FakeTrue/WELFake_Dataset.csv")
WELfake_data = WELfake_data.drop("Unnamed: 0", axis=1)
WELfake_data = WELfake_data.dropna(subset=['text'])

In [10]:
combined = pd.concat([fakenews_data, realnews_data, WELfake_data], ignore_index=True)
combined = combined.drop("subject", axis=1)
combined = combined.drop("date", axis=1)
combined = combined.dropna(subset=["title"])
combined = combined.sample(frac=0.05, random_state=42)
combined

,title,text,label
104026,N.J. Democrats divided on renewing 'Bridgegate...,(This March 30 story was corrected to note Pr...,0
49611,NO KIDDING! HERE’S WHY HILLARY SUPPORTERS Will...,Wow! The Dems are so out of touch,1
79194,Iraqi Army: US Hindering Advance on Mosul,LUCIFER in the Temple of the Dog II ‹ › GPD is...,1
51275,Britain frustrated by slow pace of Brexit talk...,"MANCHESTER, England (Reuters) - Prime Minister...",0
101910,Israeli Trump supporters open campaign office ...,JERUSALEM (Reuters) - Israeli supporters of U....,0
...,...,...,...
113761,"No, Mexico Doesn’t Have A Wall On Its Southern...","They're not crossing the Rio Grande, they're i...",1
110300,Trump Mocks Biden's Dare To Take Him 'Behind t...,"\nIn the schoolyard of American politics, Rep...",1
1801,Is Nothing Sacred? Ivanka Trump’s Passover Fa...,There are big scandals like Trump s ties to ...,0
71827,U.N. discussing resolution after Syrian govern...,UNITED NATIONS (Reuters) - The United Nations ...,0


### Model 1: CountVectorizer based on title and text of news article

In [11]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    return text

In [12]:
combined['combined_text'] = combined['title'] + ' ' + combined['text']
combined['combined_text'] = combined['combined_text'].apply(preprocess_text)
X = combined['combined_text']
y = combined['label']

In [13]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=47)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=47)

In [14]:
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_val_count = count_vectorizer.transform(X_val)
X_test_count = count_vectorizer.transform(X_test)
model = LogisticRegression()
model.fit(X_train_count, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
y_pred = model.predict(X_test_count)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.5526315789473685


### Model 2: Count Vectorizer based on clickbait of news article titles

In [16]:
def predict_label(confidence):
    if confidence < 0.166:
        return "pants-fire"
    elif confidence < 0.33:
        return "false"
    elif confidence < 0.5:
        return "barely-true"
    elif confidence < 0.666:
        return "half-true"
    elif confidence < 0.833:
        return "mostly-true" 
    else:
        return "true"

In [17]:
names = ["Linear SVM"] 

classifiers = [SVC(kernel="linear", C=0.025, probability=True)]


#Preprocess, train/test split, and 
clickbait['PreprocessedTitle'] = clickbait['headline'].apply(preprocess_text)
X_train, X_test, y_train, y_test = train_test_split(clickbait['PreprocessedTitle'], clickbait['clickbait'], test_size=0.2, random_state=42)
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)


max_score = 0.0
max_class = ''
# iterate over classifiers
for name, clf in zip(names, classifiers):
    start_time = time.time()
    clf.fit(X_train_counts, y_train)
    score = 100.0 * clf.score(X_test_counts, y_test)
    print('Classifier = %s, Score (test, accuracy) = %.2f,' %(name, score), 'Training time = %.2f seconds' % (time.time() - start_time))
    
    if score > max_score:
        clf_best = clf
        max_score = score
        max_class = name

print(80*'-' )
print('Best --> Classifier = %s, Score (test, accuracy) = %.2f' %(max_class, max_score))

#Predict probabilities for each class and get confidence
y_probabilities = clf_best.predict_proba(X_test_counts)
confidence_clickbait = y_probabilities[:, 1]

#Creating labels that compare to Liar Plus Data
predicted_labels = [predict_label(conf) for conf in confidence_clickbait]

Classifier = Linear SVM, Score (test, accuracy) = 95.67, Training time = 54.54 seconds
--------------------------------------------------------------------------------
Best --> Classifier = Linear SVM, Score (test, accuracy) = 95.67


#### Testing on Liar Plus Dataset

In [18]:
liar_plus_test = pd.read_csv('Data/Liar_plus/test.tsv', delimiter='\t', header = None, quoting=csv.QUOTE_NONE)
liar_plus_test.columns = ["ID", "Json_File_ID", "Truth_Label", "Statement", "Subject", "Speaker", "Speakers_Job", "State", "Party",
                      "Barely_True_Counts", "False_Counts", "Half_True_Counts", "Mostly_True_Counts", "Pants_On_Fire_Counts", "Context_Venue_Location", "Justification"]
liar_plus_test = liar_plus_test.drop(columns='ID')
liar_plus_test['Statement'] = liar_plus_test['Statement'].astype(str)
liar_plus_test['PreprocessedTitle'] = liar_plus_test['Statement'].apply(preprocess_text)

X_new_counts = count_vectorizer.transform(liar_plus_test['PreprocessedTitle'])

# Predict using the trained model
y_new_probabilities = clf_best.predict_proba(X_new_counts)
confidence_new = y_new_probabilities[:, 1]

# Create labels for the new data
predicted_labels_new = [predict_label(conf) for conf in confidence_new]

actual_labels = liar_plus_test['Truth_Label']
actual_labels = actual_labels.tolist()

# Compare the predicted labels to the actual labels
correct_predictions = (predicted_labels_new == actual_labels)

# Calculate accuracy
accuracy = accuracy_score(actual_labels, predicted_labels_new)
accuracy

0.1401077752117013